In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as tf_keras

In [ ]:
%%bash

cp drive/MyDrive/Colab\ Notebooks/data-files/dogs-vs-cats.zip sample_data/
unzip sample_data/dogs-vs-cats.zip -d sample_data
unzip sample_data/dogs-vs-cats/train.zip -d sample_data/dogs-vs-cats
unzip sample_data/dogs-vs-cats/test1.zip -d sample_data/dogs-vs-cats
mv sample_data/dogs-vs-cats/test1 sample_data/dogs-vs-cats/test
rm -rf sample_data/__MACOSX

In [3]:
import os
import shutil

src_base = 'sample_data/dogs-vs-cats'
dest_base = 'sample_data/dogs-vs-cats2'

if os.path.exists(dest_base): # 경로 존재여부 확인
  shutil.rmtree(dest_base) # train 경로 및 하위 경로 삭제
os.mkdir(dest_base) # 디렉터리 만들기

# train 폴더의 0 ~ 1000 : train, 1000 ~ 1500 : validation, 1500 ~ 2000 : test 세트로 구성
for start, stop, path in zip([0, 1000, 1500], [1000, 1500, 2000], ['train', 'validation', 'test']):
  os.mkdir(os.path.join(dest_base, path))
  for sub_path in ['cat', 'dog']:
    os.mkdir(os.path.join(dest_base, path, sub_path))
    for idx in np.arange(start, stop):
      fname = f'{sub_path}.{idx}.jpg'
      shutil.copy(os.path.join(src_base, 'train', fname), os.path.join(dest_base, path, sub_path, fname)) # 파일 복사

In [5]:
# 입력 자동화 도구 만들기 : 파일을 읽어서 모델에 입력 가능한 형식으로 변환하는 도구

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_generator = ImageDataGenerator(rescale=1/255) # 특정 디렉터리의 파일을 읽어서 모델의 입력데이터로 변환하는 도구
validation_data_generator = ImageDataGenerator(rescale=1/255)
test_data_generator = ImageDataGenerator(rescale=1/255)

In [6]:
# generator를 사용해서 파일 데이터 읽기
datasets = []
for path, generator in zip(['train', 'validation', 'test'],
                           [train_data_generator, validation_data_generator, test_data_generator]):
  dataset = generator.flow_from_directory(directory=f"sample_data/dogs-vs-cats2/{path}",
                                          target_size=(256, 256),
                                          batch_size=32,
                                          class_mode="binary")
  datasets.append(dataset)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
# 이미 학습 완료된 비슷한 모델 활용 ( 전이학습 )

# 1. 다른 모델의 합성곱 층이 출력한 데이터를 현재 모델의 입력으로 사용
# 2. 다른 모델과 현재 모델 결합 1 (다른 모델의 합성곱 층에 층을 추가해서 새 모델 구성 - 다른 모델의 합성곱 층은 학습하지 않음 )
# 3. 다른 모델과 현재 모델 결합 2 (다른 모델의 합성곱 층에 층을 추가해서 새 모델 구성 - 다른 모델의 합성곱 층의 일부는 학습 )

In [7]:
# 사전 학습 모델 준비

base_model = tf_keras.applications.vgg16.VGG16(include_top=False,
                                               weights='imagenet',
                                               input_shape=(256, 256, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [8]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [20]:
# 사전 학습 모델의 출력 테스트
import cv2

cat_img = cv2.imread('sample_data/dogs-vs-cats/train/cat.0.jpg', cv2.IMREAD_COLOR) # BRG
cat_img = cv2.cvtColor(cat_img, cv2.COLOR_BGR2RGB) # BGR -> RGB
cat_img = cv2.resize(cat_img, (256, 256))
print(cat_img.shape)

input = cat_img.reshape((-1, 256, 256, 3)) # 입력형식 : ( 배치크기, 데이터-shape )
print( input.shape )

output = base_model.predict(input)
print(output.shape)

(256, 256, 3)
(1, 256, 256, 3)
1/1 [==============================] - 0s 17ms/step
(1, 8, 8, 512)


In [ ]:
# 입력데이터를 받아서 사전 학습 모델에서 처리한 특성 맵 생성 함수 만들기
def get_features_and_labels_by_vgg16_base_model(dataset, cnt):
  all_features = []
  all_labels = []

  for idx, (images, labels) in enumerate(dataset):
    # features = base_model.predict(images)
    processed_images = tf_keras.applications.vgg16.preprocess_input(images) # 기본 vgg16 모델에 맞게 데이터 변형
    features = base_model.predict(processed_images) # 합성곱층을 거친 데이터
    all_features.append(features)
    all_labels.append(labels)

    if idx == cnt:
      break

  return np.concatenate(all_features), np.concatenate(all_labels)


In [ ]:
# 훈련데이터, 검증데이터, 테스트 데이터에 대해 사전 학습 모델에서 처리한 특성 맵 생성
train_features, train_labels = get_features_and_labels_by_vgg16_base_model(datasets[0], 100)
validation_features, validation_labels = get_features_and_labels_by_vgg16_base_model(datasets[1], 50)
test_features, test_labels = get_features_and_labels_by_vgg16_base_model(datasets[2], 50)

In [35]:
train_features.shape, train_labels.shape, test_features.shape, test_labels.shape

((3200, 8, 8, 512), (3200,), (1608, 8, 8, 512), (1608,))

In [37]:
# 사전 학습 모델을 통해 만든 특성맵 데이터를 사용하는 판별 모델 구성

# model1 = tf_keras.Sequential([
#     tf_keras.layers.Input(shape=(8, 8, 512)),
#     tf_keras.layers.Flatten(),
#     tf_keras.layers.Dense(units=256, activation="relu"),
#     tf_keras.layers.Dense(units=1, activation="sigmoid")
# ])

input = tf_keras.layers.Input(shape=(8, 8, 512))
x = tf_keras.layers.Flatten()(input)
x = tf_keras.layers.Dense(units=256, activation="relu")(x)
output = tf_keras.layers.Dense(units=1, activation="sigmoid")(x)

model1 = tf_keras.models.Model(input, output)

In [39]:
model1.compile(optimizer="adam",
               loss="binary_crossentropy",
               metrics=['accuracy'])

In [40]:
history = model1.fit(train_features, train_labels,
                     batch_size=32,
                     epochs=20,
                     validation_data=(validation_features, validation_labels))

Epoch 1/20
100/100 [==============================] - 4s 16ms/step - loss: 1.2329 - accuracy: 0.4931 - val_loss: 0.6853 - val_accuracy: 0.5299
Epoch 2/20
100/100 [==============================] - 1s 7ms/step - loss: 0.6903 - accuracy: 0.5484 - val_loss: 0.6832 - val_accuracy: 0.4975
Epoch 3/20
100/100 [==============================] - 1s 9ms/step - loss: 0.6918 - accuracy: 0.5381 - val_loss: 0.7040 - val_accuracy: 0.5193
Epoch 4/20
100/100 [==============================] - 1s 9ms/step - loss: 0.6973 - accuracy: 0.5263 - val_loss: 0.6783 - val_accuracy: 0.5143
Epoch 5/20
100/100 [==============================] - 1s 7ms/step - loss: 0.6817 - accuracy: 0.5603 - val_loss: 0.6671 - val_accuracy: 0.5920
Epoch 6/20
100/100 [==============================] - 1s 9ms/step - loss: 0.6671 - accuracy: 0.5913 - val_loss: 0.7318 - val_accuracy: 0.4950
Epoch 7/20
100/100 [==============================] - 1s 9ms/step - loss: 0.6749 - accuracy: 0.5763 - val_loss: 0.6565 - val_accuracy: 0.6275
Epoch